Building a Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 512)
        self.linear2 = nn.Linear(512, 512)
        self.linear3 = nn.Linear(512, 10)
        

    def forward(self, x):
        x = self.flatten(x)

        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        
        return x

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print its structure.

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=10, bias=True)
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.

In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


Model Layers
-------------------------

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using its stored weights and biases.


In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there's other activations to introduce non-linearity in your model.



In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)

Before ReLU: tensor([[-0.2794, -0.2937,  0.2934, -0.0954,  0.1049,  0.3690, -0.6407,  0.0720,
         -0.2886,  0.1894,  0.2220, -0.1684,  0.3662,  0.1397, -0.3651, -0.3935,
          0.1049, -0.0100, -0.1236,  0.6182],
        [-0.1044, -0.1074,  0.3470,  0.0214,  0.5537,  0.1624, -0.8189, -0.1113,
         -0.3240,  0.2411,  0.0442,  0.0969,  0.4958,  0.3376, -0.4703,  0.0303,
          0.0237, -0.0820,  0.1643,  0.6427],
        [-0.1677, -0.3927,  0.1644,  0.2493,  0.5144,  0.3865, -0.8020,  0.1725,
         -0.4361,  0.1186,  0.4164,  0.6004,  0.6661,  0.5763, -0.3688, -0.1836,
          0.1766,  0.0280,  0.2708,  0.1698]], grad_fn=<AddmmBackward0>)




In [11]:
print(f"After ReLU: {hidden1}")

After ReLU: tensor([[0.0000, 0.0000, 0.2934, 0.0000, 0.1049, 0.3690, 0.0000, 0.0720, 0.0000,
         0.1894, 0.2220, 0.0000, 0.3662, 0.1397, 0.0000, 0.0000, 0.1049, 0.0000,
         0.0000, 0.6182],
        [0.0000, 0.0000, 0.3470, 0.0214, 0.5537, 0.1624, 0.0000, 0.0000, 0.0000,
         0.2411, 0.0442, 0.0969, 0.4958, 0.3376, 0.0000, 0.0303, 0.0237, 0.0000,
         0.1643, 0.6427],
        [0.0000, 0.0000, 0.1644, 0.2493, 0.5144, 0.3865, 0.0000, 0.1725, 0.0000,
         0.1186, 0.4164, 0.6004, 0.6661, 0.5763, 0.0000, 0.0000, 0.1766, 0.0280,
         0.2708, 0.1698]], grad_fn=<ReluBackward0>)


#### nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in [$-\infty$, $\infty$] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.


In [12]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.

In [13]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=10, bias=True)
)


Layer: linear1.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0286, -0.0222, -0.0085,  ..., -0.0117,  0.0225, -0.0329],
        [ 0.0013, -0.0336,  0.0289,  ..., -0.0042,  0.0051, -0.0356]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear1.bias | Size: torch.Size([512]) | Values : tensor([0.0054, 0.0191], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0261,  0.0263, -0.0060,  ..., -0.0403,  0.0057,  0.0398],
        [ 0.0183,  0.0251, -0.0046,  ..., -0.0404, -0.0103, -0.0207]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear2.bias | Size: torch.Size([512]) | Values : tensor([0.0318, 0.0110]

## Other ways to build NNs

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like ``seq_modules``.

In [14]:
class NN_Sequential(nn.Module):
    def __init__(self):
        super(NN_Sequential, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [15]:
model = NN_Sequential().to(device)
print(model)

NN_Sequential(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### nn.ModuleList

Holds submodules in a list.

[ModuleList](https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html) can be indexed like a regular Python list, but modules it contains are properly registered, and will be visible by all ``Module`` methods.

In [16]:
class NN_ModuleList(nn.Module):
    def __init__(self):
        super(NN_ModuleList, self).__init__()
        self.flatten = nn.Flatten()

        self.linear_layers = nn.ModuleList()
        self.linear_layers.append(nn.Linear(28*28, 512))
        self.linear_layers.append(nn.Linear(512, 512))

        self.out_layer = nn.Linear(512, 10)


    def forward(self, x):
        x = self.flatten(x)

        for layer in self.linear_layers:
          x = self.layer(x)
          x = F.relu(x)

        x = self.out_layer(x)
        return x

In [18]:
model = NN_ModuleList().to(device)
print(model)

NN_ModuleList(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_layers): ModuleList(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
  )
  (out_layer): Linear(in_features=512, out_features=10, bias=True)
)


## Exercise
Create neural network with 3 hidden layers with 32 neurons in each hidden layer. Note that the input size is $28×28=784$ and we have 10 digits/classes.

![ffnn](https://imgs.search.brave.com/ooSheVMBI1K75nCskOlS2h4EYshOxW1JonxIZmqF6q8/rs:fit:790:393:1/g:ce/aHR0cHM6Ly93d3cu/cmVzZWFyY2hnYXRl/Lm5ldC9wcm9maWxl/L1RlZGR5LUd1bmF3/YW4vcHVibGljYXRp/b24vMzIzNzA0OTEz/L2ZpZ3VyZS9maWcz/L0FTOjY2OTI4MjEx/NTUyMjU4MUAxNTM2/NTgwNjk5NTA3L0Rl/ZXAtRmVlZGZvcndh/cmQtTmV1cmFsLU5l/dHdvcmstU3RydWN0/dXJlLnBuZw)